In [1]:
import numpy as np
import pandas as pd
import pickle
import random
import matplotlib.pyplot as plt

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [14]:
# best_model_path = '../../tr_models/classifierX3_best.pth'
# last_model_path = '../../tr_models/classifierX3_last.pth'
# plot_vals_path = '../../outputs/classifierX3_plotres.pkl'

tr_labels = pd.read_csv('../../data/small_ntu_frames/train_labels', header=None)
v_labels = pd.read_csv('../../data/small_ntu_frames/val_labels', header=None)

with open('../../data/small_ntu_frames/train_data.pkl', 'rb') as f:
    tr_dict = pickle.load(f)
with open('../../data/small_ntu_frames/val_data.pkl', 'rb') as f:
    v_dict = pickle.load(f)

In [15]:
def flatten_frames(arr_3d):
    n_frames = arr_3d.shape[0]
    temp_arr = np.zeros((n_frames, 48))
    for i in range(n_frames):
        arr_3d[i, :, :] = arr_3d[i, :, :] - arr_3d[i, 7, :]
        norz = np.linalg.norm(arr_3d[1, 6, :] - arr_3d[i, 7, :])
        arr_3d[i, :, :] /= 1.0*norz
        temp_arr[i, :] = np.reshape(arr_3d[i, :, :], (1, -1))
    return temp_arr

def flatten_dict(d_dict):
    for i in d_dict:
        d_dict[i] = flatten_frames(d_dict[i])
    return d_dict

In [16]:
tr_d = flatten_dict(tr_dict)
v_d = flatten_dict(v_dict)

In [17]:
def data_gen(d_dict, d_labels):
    while(True):
        d_id = random.sample(list(d_dict), 1)[0]
        databatch = d_dict[d_id]
        if 'train' in d_id:
            idx = int(d_id.replace('train', ''))
            label = d_labels.iloc[idx, 1]
        elif 'val' in d_id:
            idx = int(d_id.replace('val', ''))
            label = d_labels.iloc[idx, 1]
        yield databatch,label

def val_data_gen(d_dict, d_labels):
    counter = 0
    n_s = len(d_dict)
    while(counter<n_s):
        d_id = list(d_dict)[counter]
        databatch = d_dict[d_id]
        if 'val' in d_id:
            idx = int(d_id.replace('val', ''))
            label = d_labels.iloc[idx, 1]
        counter += 1
        counter = counter % n_s
        yield databatch,label

In [18]:
Dtr = data_gen(tr_d, tr_labels)
Dv = data_gen(v_d, v_labels)

In [31]:
#action LSTM
class LSTMClassifier(nn.Module):
    def __init__(self, joints_dim, hidden_dim, label_size, batch_size, num_layers, kernel_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        joints_dim2d = joints_dim - 16
        
        self.lstm3 = nn.LSTM(joints_dim, hidden_dim, num_layers=self.num_layers)
        self.conv1_3 = nn.Conv1d(1, 1, kernel_size, stride=1, padding=1)
        
        self.lstm2_1 = nn.LSTM(joints_dim2d, hidden_dim, num_layers=self.num_layers)
        self.conv1_2_1 = nn.Conv1d(1, 1, kernel_size, stride=1, padding=1)
        self.lstm2_2 = nn.LSTM(joints_dim2d, hidden_dim, num_layers=self.num_layers)
        self.conv1_2_2 = nn.Conv1d(1, 1, kernel_size, stride=1, padding=1)
        self.lstm2_3 = nn.LSTM(joints_dim2d, hidden_dim, num_layers=self.num_layers)
        self.conv1_2_3 = nn.Conv1d(1, 1, kernel_size, stride=1, padding=1)
        
#         self.conv1_1 = nn.Conv1d(4, 2, kernel_size, stride=1, padding=1) #for kernel size=3
#         self.conv1_2 = nn.Conv1d(2, 1, kernel_size, stride=1, padding=1) #for kernel size=3
        
        self.hidden3 = self.init_hidden3()
        self.hidden2_1 = self.init_hidden2_1()
        self.hidden2_2 = self.init_hidden2_2()
        self.hidden2_3 = self.init_hidden2_3()
        
        self.hidden2label = nn.Linear(hidden_dim, label_size)
    
    def init_hidden3(self):
        # the first is the hidden h
        # the second is the cell  c
        return (autograd.Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_dim).cuda()),
                autograd.Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_dim).cuda()))
    def init_hidden2_1(self):
        # the first is the hidden h
        # the second is the cell  c
        return (autograd.Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_dim).cuda()),
                autograd.Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_dim).cuda()))
    def init_hidden2_2(self):
        # the first is the hidden h
        # the second is the cell  c
        return (autograd.Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_dim).cuda()),
                autograd.Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_dim).cuda()))
    def init_hidden2_3(self):
        # the first is the hidden h
        # the second is the cell  c
        return (autograd.Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_dim).cuda()),
                autograd.Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_dim).cuda()))
    
    
    def forward(self, joints3d_vec):
        x3 = joints3d_vec
        x2 = x3.view(-1, 16, 3)
        x2_1 = x2[:,:,1:3].contiguous().view(-1, 1, 32)
        x2_2 = x2[:,:,0:2].contiguous().view(-1, 1, 32)
        x2_3 = x2[:,:,[0,2]].contiguous().view(-1, 1, 32)
        
        lstm_out3, self.hidden3 = self.lstm3(x3, self.hidden3)
        lstm_out2_1, self.hidden2_1 = self.lstm2_1(x2_1, self.hidden2_1)
        lstm_out2_2, self.hidden2_2 = self.lstm2_2(x2_2, self.hidden2_2)
        lstm_out2_3, self.hidden2_3 = self.lstm2_3(x2_3, self.hidden2_3)
        
        t3 = lstm_out3[-1].view(self.batch_size,1,-1)
        t2_1 = lstm_out2_1[-1].view(self.batch_size,1,-1)
        t2_2 = lstm_out2_2[-1].view(self.batch_size,1,-1)
        t2_3 = lstm_out2_3[-1].view(self.batch_size,1,-1)

        y3 = self.conv1_3(t3)
        y2_1 = self.conv1_2_1(t2_1)
        y2_2 = self.conv1_2_2(t2_2)
        y2_3 = self.conv1_2_3(t2_3)
        
        y3 += y2_1+y2_2+y2_3
        
        y3 = y3.contiguous().view(-1, self.hidden_dim)        
        y  = self.hidden2label(y3)
        
        log_probs = F.softmax(y, dim=1)
        return log_probs
#instanstiating a model
model0 = LSTMClassifier(48, 128, 8, 1, 2, 3)
#to do stuff in CUDA
model0 = model0.cuda()

In [34]:
model0.load_state_dict(torch.load('../../tr_models/classifierX3_best.pth'))

In [19]:
v_d['val132']

array([[ -3.80250983e-02,   4.30356972e-01,   1.40913907e-02, ...,
          1.26750328e-02,   1.26948375e-01,  -3.62744387e-02],
       [ -1.85335036e-01,   2.15934623e+00,   1.44080815e-01, ...,
         -1.23556691e-01,   6.15852880e-01,  -1.86213126e-01],
       [ -1.20000000e+01,   1.39812500e+02,   6.78567505e+00, ...,
         -8.00000000e+00,   4.38750000e+01,  -1.03040466e+01],
       ..., 
       [ -1.60000000e+01,   1.39750000e+02,   7.62510681e+00, ...,
         -4.00000000e+00,   3.19375000e+01,  -1.28982391e+01],
       [ -1.60000000e+01,   1.35750000e+02,   6.28524780e+00, ...,
          4.00000000e+00,   4.00625000e+01,  -9.56851196e+00],
       [ -1.60000000e+01,   1.39750000e+02,   2.14558411e+00, ...,
          1.20000000e+01,   4.01875000e+01,  -7.95022583e+00]])

In [20]:
X = autograd.Variable(torch.from_numpy(v_d['val132']).float()).cuda()
X1 = X.view(len(X), 1, -1)

In [25]:
X1

Variable containing:
(0 ,.,.) = 
   -0.0380    0.4304    0.0141  ...     0.0127    0.1269   -0.0363

(1 ,.,.) = 
   -0.1853    2.1593    0.1441  ...    -0.1236    0.6159   -0.1862

(2 ,.,.) = 
  -12.0000  139.8125    6.7857  ...    -8.0000   43.8750  -10.3040
...

(49,.,.) = 
  -16.0000  139.7500    7.6251  ...    -4.0000   31.9375  -12.8982

(50,.,.) = 
  -16.0000  135.7500    6.2852  ...     4.0000   40.0625   -9.5685

(51,.,.) = 
  -16.0000  139.7500    2.1456  ...    12.0000   40.1875   -7.9502
[torch.cuda.FloatTensor of size 52x1x48 (GPU 0)]

In [36]:
y_hat = model0(X1)
y_pred = np.argmax(y_hat.data.cpu().numpy())
print('prediction is class : {}'.format(y_pred))

prediction is class : 2
